In [1]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
tf.enable_eager_execution(config=config)

import os
from multiprocessing import Process, Manager
from random import shuffle

/home/james/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
train_dir = "train/"
classes = os.listdir(train_dir)
classes = sorted(classes)
num_classes = len(classes)

def list_all(class_num, class_name, shared, *args):
    class_dir = train_dir + class_name+ "/"
    file_list = os.listdir(class_dir)
        
    class_list = manager.list(shared[class_num])

    for file in file_list:
        file_dir = class_dir + file

        image = tf.read_file(file_dir)
        image = tf.image.decode_image(image)
        if image.shape[2] == 3:
            class_list.append([file_dir, class_num])
            
    shared[class_num] = class_list

            
manager = Manager()
shared = manager.list([[]]*num_classes)
p = []
for class_num, class_name in enumerate(classes):
    p.append(Process(target=list_all, args=(class_num, class_name, shared)))
    p[class_num].start()

for i in range(class_num):
    p[i].join()

all_list = [item for sublist in shared for item in sublist]
shuffle(all_list)
x, y = zip(*all_list)

In [3]:
print(len(all_list))

train_x = x[:int(0.8*len(x))]
train_y = y[:int(0.8*len(y))]

test_x = x[int(0.8*len(x)):]
test_y = y[int(0.8*len(y)):]

#print(len(train_x), len(test_x))

4726


In [4]:
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def write_tfrecord(name, x, y):
    
    tfrecord_path = name + ".tfrecords"
    writer = tf.python_io.TFRecordWriter(tfrecord_path)
    
    for i in range(len(x)):
        file_dir_byte = bytes(x[i], 'utf-8')
        example = tf.train.Example(
            features=tf.train.Features(
                feature={
                    'x': _bytes_feature(file_dir_byte),
                    'y': _int64_feature(int(y[i]))
                }))
        writer.write(example.SerializeToString())
        
    writer.close()
    
write_tfrecord("train",train_x,train_y)
write_tfrecord("test",test_x, test_y)